# Actividad 2 Analisis de Informacion No Estructurada

# - Javier Gilabert Sabater

## 1. Lea el contenido del fichero csv en un DataFrame.

In [24]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [25]:
df = pd.read_csv("spam.csv", sep=',')
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


------

## 2. Realice el pre-procesamiento que considere necesario. Puede utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, y contestar a las preguntas del punto 6.

In [26]:
corpus = df.text.to_list()

def limpiar_texto(texto):
    if not isinstance(texto, str):
        raise ValueError("El argumento 'texto' debe ser una cadena de texto.")
    
    # Eliminar caracteres especiales y números
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    
    # Convertir a minúsculas
    texto = texto.lower()
    
    # Tokenizar el texto
    tokens = word_tokenize(texto)
    
    # Eliminar stop words
    stop_words = set(stopwords.words('english'))
    tokens_filtrados = [word for word in tokens if word not in stop_words]
    
    # Unir tokens nuevamente en un string
    texto_limpiado = ' '.join(tokens_filtrados)
    
    return texto_limpiado

def limpiar_corpus(corpus):
    corpus_limpiado = []
    
    for email in corpus:
        try:
            email_limpiado = limpiar_texto(email)
            corpus_limpiado.append(email_limpiado)
        except ValueError as e:
            print(f"Error: {e}")
            corpus_limpiado.append('error')
    
    return corpus_limpiado

# Llamar a la función para limpiar el corpus
corpus_limpiado = limpiar_corpus(corpus)

In [27]:
df['corpus_limpiado'] = df['text'].apply(lambda x: limpiar_texto(x))
df

,label,text,corpus_limpiado
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,nd time tried contact u u pound prize claim ea...
5568,ham,Will Ì_ b going to esplanade fr home?,b going esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood soany suggestions
5570,ham,The guy did some bitching but I acted like i'd...,guy bitching acted like id interested buying s...


------

## 3. Convierta el corpus de documentos en una matriz TF-idf.

In [28]:
# Crear una instancia de TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Ajustar el vectorizador a los argumentos preprocesados y transformar los datos
X_tfidf = tfidf_vectorizer.fit_transform(corpus_limpiado)

# Convertir la matriz X_tfidf a un DataFrame de pandas
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=corpus_limpiado)

# df_tfidf ahora contiene la representación TF-IDF en formato DataFrame
df_tfidf

,aa,aah,aaniye,aaooooright,aathilove,aathiwhere,ab,abbey,abdomen,abeg,...,zeros,zf,zhong,zindgi,zoe,zogtorius,zoom,zouk,zs,zyada
go jurong point crazy available bugis n great world la e buffet cine got amore wat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ok lar joking wif u oni,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
free entry wkly comp win fa cup final tkts st may text fa receive entry questionstd txt ratetcs apply overs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u dun say early hor u c already say,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
nah dont think goes usf lives around though,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nd time tried contact u u pound prize claim easy call p per minute btnationalrate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b going esplanade fr home,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pity mood soany suggestions,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
guy bitching acted like id interested buying something else next week gave us free,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


-------

## 4. Divida en un subconjunto de entrenamiento y otro de evaluación.

In [29]:
random_num = 100
df_train, df_test = train_test_split(df, test_size=0.3, random_state=random_num)

-----

## 5. Llegados a este punto, realice modelos de entrenamiento al menos con algoritmos de clasificador bayesiano ingenuo, máquinas SVM y un modelo basado en árbol de decisión. Obtenga resultados de accuracy de la clasificación, así como las matrices de confusión para los tres modelos.

### Clasificador Bayesiano Ingenuo

In [30]:
# Crear el modelo usando un pipeline con TF-IDF y el clasificador Naive Bayes o Bayesiano Ingenuo
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Entrenar el modelo con el conjunto de entrenamiento
model.fit(df_train['corpus_limpiado'], df_train['label'])

# Realizar predicciones en el conjunto de evaluación
predictions = model.predict(df_test['corpus_limpiado'])

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(df_test['label'], predictions)
conf_matrix = confusion_matrix(df_test['label'], predictions)
classification_rep = classification_report(df_test['label'], predictions)

# Mostrar los resultados
print("Naive Bayes:")
print(f'Accuracy: {accuracy:.2f}')
print('\nConfusion Matrix:') 
print(conf_matrix)
print('\nClassification Report:')
print(classification_rep)

Naive Bayes:
Accuracy: 0.97

Confusion Matrix:
[[1459    0]
 [  56  157]]

Classification Report:
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1459
        spam       1.00      0.74      0.85       213

    accuracy                           0.97      1672
   macro avg       0.98      0.87      0.91      1672
weighted avg       0.97      0.97      0.96      1672



- Con este modelo se ha alcanzado una alta precisión del 97%, con un alto recall para los mensajes NO spam (ham) del 100%, pero con un bajo recall para los mensajes spam del 74%.
- En la matriz de confusion podemos observar 56 (FN) Falsos Negativos que fueron identificados como ham, siendo realmente spam.

### Maquinas SVM

In [31]:
# Crear el modelo usando un pipeline con TF-IDF y el clasificador Naive Bayes o Bayesiano Ingenuo
svm_model = make_pipeline(TfidfVectorizer(), SVC())

# Entrenar el modelo con el conjunto de entrenamiento
svm_model.fit(df_train['corpus_limpiado'], df_train['label'])

# Realizar predicciones en el conjunto de evaluación
svm_predictions = svm_model.predict(df_test['corpus_limpiado'])

# Evaluar el rendimiento del modelo
svm_accuracy = accuracy_score(df_test['label'], svm_predictions)
svm_conf_matrix = confusion_matrix(df_test['label'], svm_predictions)
svm_classification_rep = classification_report(df_test['label'], svm_predictions)

# Mostrar los resultados
print('Maquinas SVM:')
print(f'Accuracy: {svm_accuracy:.2f}')
print('\nConfusion Matrix:') 
print(svm_conf_matrix)
print('\nClassification Report:')
print(svm_classification_rep)

Maquinas SVM:
Accuracy: 0.98

Confusion Matrix:
[[1455    4]
 [  35  178]]

Classification Report:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1459
        spam       0.98      0.84      0.90       213

    accuracy                           0.98      1672
   macro avg       0.98      0.92      0.94      1672
weighted avg       0.98      0.98      0.98      1672



- Con este modelo se ha alcanzado una precisión aun mayor que con el modelo anterior del 98%, con un alto recall para los mensajes NO spam (ham) del 100%, pero con un bajo recall para los mensajes spam, aunque mayor que con el bayesiano del 84%.
- En la matriz de confusion podemos observar 35 (FN) Falsos Negativos que fueron identificados como ham, siendo realmente spam. Y 4 (FP) Falsos Positivos que fueron identificados como spam, siendo realmente ham.

### Arboles de Decision

In [32]:
# Crear el modelo usando un pipeline con TF-IDF y el clasificador Naive Bayes o Bayesiano Ingenuo
tree_model = make_pipeline(TfidfVectorizer(), DecisionTreeClassifier(random_state=random_num))

# Entrenar el modelo con el conjunto de entrenamiento
tree_model.fit(df_train['corpus_limpiado'], df_train['label'])

# Realizar predicciones en el conjunto de evaluación
tree_predictions = tree_model.predict(df_test['corpus_limpiado'])

# Evaluar el rendimiento del modelo
tree_accuracy = accuracy_score(df_test['label'], tree_predictions)
tree_conf_matrix = confusion_matrix(df_test['label'], tree_predictions)
tree_classification_rep = classification_report(df_test['label'], tree_predictions)

# Mostrar los resultados
print('Arbol de Decision:')
print(f'Accuracy: {tree_accuracy:.2f}')
print('\nConfusion Matrix:') 
print(tree_conf_matrix)
print('\nClassification Report:')
print(tree_classification_rep)

Arbol de Decision:
Accuracy: 0.96

Confusion Matrix:
[[1434   25]
 [  40  173]]

Classification Report:
              precision    recall  f1-score   support

         ham       0.97      0.98      0.98      1459
        spam       0.87      0.81      0.84       213

    accuracy                           0.96      1672
   macro avg       0.92      0.90      0.91      1672
weighted avg       0.96      0.96      0.96      1672



- Con este modelo se ha alcanzado una precisión alta aunque la mas baja de todos los modelos del 96%, con un alto recall para los mensajes NO spam (ham) del 98%, mas bajo que cualquier modelo, pero con un bajo recall para los mensajes spam del 81%, mas bajo que el modelo SVM.
- En la matriz de confusion podemos observar 40 (FN) Falsos Negativos que fueron identificados como ham, siendo realmente spam. Y 25 (FP) Falsos Positivos que fueron identificados como spam, siendo realmente ham.

### En conclusion, comparando los tres modelos, podemos decir que el mejor modelo de todos es el SVM, al ser el mas preciso de los tres en todos los ambitos, con mas precision y recall que cualquiera de los otros modelos. Aunque si que hay que tener en cuenta que el bayesiano no tenia falsos positivos, mientras que el svm tiene 4 FP, aunque la cantidad de FN tambien la tiene el SVM, asi que este seria el modelo a escoger.

-----

## 6. Conteste a las siguientes preguntas basándote en evidencias de código. ¿Tiene influencia en el resultado final el número máximo de features a utilizar? Prueba al menos dos números de features diferentes para los tres algoritmos y mide los resultados. ¿Modifica el resultado si no se eliminan las stop words? Pruébalo para los tres algoritmos y mide los resultados.

## - Influencia del numero maximo de features a utilizar

### Clasificador Bayesiano Ingenuo

In [41]:
max_features_values = [100, 1000]

for max_features in max_features_values:
    model = make_pipeline(TfidfVectorizer(max_features=max_features), MultinomialNB())

    model.fit(df_train['corpus_limpiado'], df_train['label'])

    predictions = model.predict(df_test['corpus_limpiado'])

    accuracy = accuracy_score(df_test['label'], predictions)
    print(f'Max Features: {max_features}, Accuracy: {accuracy:.2f}')

Max Features: 100, Accuracy: 0.95
Max Features: 1000, Accuracy: 0.98


### Maquinas SVM

In [42]:
max_features_values = [100, 1000]

for max_features in max_features_values:
    model = make_pipeline(TfidfVectorizer(max_features=max_features), SVC())

    model.fit(df_train['corpus_limpiado'], df_train['label'])

    predictions = model.predict(df_test['corpus_limpiado'])

    accuracy = accuracy_score(df_test['label'], predictions)
    print(f'Max Features: {max_features}, Accuracy: {accuracy:.2f}')

Max Features: 100, Accuracy: 0.96
Max Features: 1000, Accuracy: 0.98


### Arboles de Decision

In [43]:
max_features_values = [100, 1000]

for max_features in max_features_values:
    model = make_pipeline(TfidfVectorizer(max_features=max_features), DecisionTreeClassifier(random_state=random_num))

    model.fit(df_train['corpus_limpiado'], df_train['label'])

    predictions = model.predict(df_test['corpus_limpiado'])

    accuracy = accuracy_score(df_test['label'], predictions)
    print(f'Max Features: {max_features}, Accuracy: {accuracy:.2f}')

Max Features: 100, Accuracy: 0.94
Max Features: 1000, Accuracy: 0.96


### En cuanto al numero de features podemos ver que afecta 100%, ya que para los tres modelos una mayor cantidad de features incrementa el accuracy, para 100 features es mas bajo en todos los casos que con 1000 features.

## - Influencia de no usar stopwords

### Clasificador Bayesiano Ingenuo

In [50]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

model.fit(df_train['text'], df_train['label'])

predictions = model.predict(df_test['text'])

accuracy = accuracy_score(df_test['label'], predictions)
print(f'No stopwords Accuracy: {accuracy:.2f}')

No stopwords Accuracy: 0.96


### Maquina SVM

In [51]:
model = make_pipeline(TfidfVectorizer(), SVC())

model.fit(df_train['text'], df_train['label'])

predictions = model.predict(df_test['text'])

accuracy = accuracy_score(df_test['label'], predictions)
print(f'No stopwords Accuracy: {accuracy:.2f}')

No stopwords Accuracy: 0.98


### Arboles de Decision

In [52]:
model = make_pipeline(TfidfVectorizer(), DecisionTreeClassifier())

model.fit(df_train['text'], df_train['label'])

predictions = model.predict(df_test['text'])

accuracy = accuracy_score(df_test['label'], predictions)
print(f'No stopwords Accuracy: {accuracy:.2f}')

No stopwords Accuracy: 0.96


### En cuanto a las stopwords podemos observar que al unico modelo al que le afecta el uso de stopwords es al bayesiano, ya que en el ejercicio 5 hemos conseguido una precision del 97% y aqui en cambio tenemos una precision del 96%. Pero en terminos general no parace afectar al mismo nivel que el numero de features que si que afecta mas la precision, ya que para el SVM y el arbol de decision nos sale la misma precision tanto con stopwords como sin stopwords

------

## 7. Imagínese que este entregable es una labor que le han solicitado en un entorno profesional, y que tiene que entregar esta documentación para comentar lo que ha descubierto (datos de entrada, rendimiento de los modelos, o cualquier descubrimiento que pueda ser importante). Comente los resultados obtenidos.

### Mirando los datos de entrada encontramos dos columnas, "text" incluyendo el texto de todos los emails y "label" indicando si el email es spam o no (ham). En cuanto al rendimiento de los modelos, se han probado tres modelos diferentes, un clasificador bayesiano ingenuo, una maquina de soporte vectorial (SVM) y un arbol de decision, adjudicando para esta tarea finalmente la maquina de soporte vectorial (SVM) debido a una mayor precision y un mayor recall. Finalmente hemos hecho unas pruebas para comprobar si el numero de features y el uso de stopwords alteraba de alguna manera la seleccion del modelo a usar. Con lo que hemos concluido que el numero de features si que afecta a la precision del modelo y el uso de stopwords solo afecta al modelo bayesiano. Como conclusion final, hemos conseguido una precision del 98% para identificar que emails son spam con el uso del modelo SVM, lo cual es una precision alta.

-----